# Experiment 1
This notebook contains the code for the first experiment of the paper. In this experiment, we train a B2BNet model that first encodes the temporal dimensions of the EEG data and then its spatial dimensions. It does not include any B2B or classification heads.

In [13]:
%reload_ext autoreload
%autoreload 3

import torch
import torch.nn.functional as F
from torchinfo import summary
import pytorch_lightning as pl
from src.b2bnet import B2BNetSpaceTimeModel, OtkaDataModule, OtkaTimeDimSplit
from pytorch_lightning.loggers import TensorBoardLogger

In [7]:
# Parameters
segment_size = 120 * 3 # 3sec
batch_size = 256
n_channels = 59
space_embedding_dim = 8
time_embedding_dim = 16
max_epochs = 1000

In [20]:
datamodule = OtkaTimeDimSplit(segment_size=segment_size, batch_size=batch_size, b2b_data=None,
                              data_mode='crop')

model = B2BNetSpaceTimeModel(
    n_channels=n_channels, space_embedding_dim=space_embedding_dim,
    time_embedding_dim=time_embedding_dim, kernel_size=1, n_subjects=52, classifier=False,
    encoder_arch='autoencoder', b2b=None)

trainer = pl.Trainer(max_epochs=max_epochs, accelerator='cpu', log_every_n_steps=1, deterministic='warn')

trainer.fit(model, datamodule=datamodule)
trainer.save_checkpoint(f'models/b2bnet_segment-{segment_size}.ckpt')

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/homebrew/Caskroom/miniforge/base/envs/b2bnet/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(

  | Name     | Type                 | Params
--------------------------------------------------
0 | baseline | SpaceTimeAutoEncoder | 4.7 K 
--------------------------------------------------
4.7 K     Trainable params
0         Non-trainable params
4.7 K     Total params
0.019     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/b2bnet/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/homebrew/Caskroom/miniforge/base/envs/b2bnet/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/b2bnet/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [33]:
from torch import nn



model_ckpt = B2BNetSpaceTimeModel.load_from_checkpoint("lightning_logs/version_112/checkpoints/epoch=0-step=16.ckpt")

x, y, subject_ids, x_b2b, y_b2b, y_cls = datamodule.val_dataset[:]
# embedding, y_hat, y_b2b_hat, embedding_b2b, y_cls_hat = model_ckpt(x, x_b2b)
h, _ = model_ckpt.baseline(x)

classifier_model = nn.Linear(h.shape[-1], 2)

torch.Size([1, 1683, 16])